In [ ]:
import pandas as pd
import json
from pprint import pformat
import sqlite3
#
from functools import wraps
import time


def timeit(func):
    @wraps(func)
    def timeit_wrapper(*args, **kwargs):
        start_time = time.perf_counter()
        result = func(*args, **kwargs)
        end_time = time.perf_counter()
        total_time = end_time - start_time
        print(f'Function {func.__name__} Took {total_time:.4f} seconds')
        return result
    return timeit_wrapper

In [ ]:
@timeit
def get_set_from_json(file_name:str):
    with open(file_name) as f:
        data = json.load(f)

    lotr_set = data['data']['cards']

    return lotr_set

lotr_set = get_set_from_json('ltr.json')

In [ ]:
@timeit
def get_all_keys(json_array:list):
    all_keys = set().union(*(d.keys() for d in json_array))
    all_keys = sorted(all_keys)
    return all_keys

all_keys = get_all_keys(lotr_set)

In [ ]:

for card in lotr_set:
    for key in card.keys():
        if key not in all_keys:
            continue
        elif card[key] is None:
            print(f'Key {key} is None')
            continue
        elif isinstance(card[key], list):
            print(f'Key {key} is a list')
            continue

In [ ]:
print(len(all_keys),type(all_keys), all_keys)

In [ ]:
drop_list = ['artist','availability','borderColor','colors','colorIdentity',
    'convertedManaCost','edhrecRank','colorIdentity','foreignData',
    'frameVersion','isStarter','language','layout','legalities',
    'manaCost','manaValue','printings','securityStamp','setCode',
    'power', 'toughness', 'edhrecSaltiness','isReprint','flavorName',
    'hasAlternativeDeckLimit',]

@timeit
def drop_keys(all_keys:list, keys_to_drop:list):
    for key in keys_to_drop:
        print(f'Dropping {key}')
        try:
            all_keys.remove(key)
        except:
            print(f'Key {key} not found')
    return all_keys

all_keys = drop_keys(all_keys, drop_list)
print(len(all_keys),type(all_keys), all_keys)

In [ ]:
@timeit
def get_dict_of_arrays(all_keys:list, json_array):
    _dict = {}
    for key in all_keys:
        _dict[key] = [card[key] if key in card else None for card in json_array]
    return _dict

lotr = get_dict_of_arrays(all_keys, lotr_set)
print(lotr)

In [ ]:
for key in lotr:
    print(key, len(lotr[key]))

In [ ]:
@timeit
for key in lotr:
    print(key, len(lotr[key]))

In [ ]:
# IGNORAR ISSO
@timeit
def get_lists_and_dicts(json_array:list, keys_to_replicate:list) -> [list, dict]:
    _lists = {}
    _dicts = {}

    for item in json_array:
        for key in item:
            if isinstance(item[key], list):
                if key not in _lists:
                    _lists[key] = []
                for i in item[key]:
                    new = {}
                    new[key] = item[key]
                    for k in keys_to_replicate:
                        new[k] = item[k]
                    _lists[key].append(new)
                new = {}
                for k in keys_to_replicate:
                    new[k] = item[k]
                
                _lists[key].append(new)
            elif isinstance(item[key], dict):
                if key not in _dicts:
                    _dicts[key] = []
                new = {}
                for k in keys_to_replicate:
                    new[k] = item[k]
                new[key] = item[key]
                _lists[key].append(new)
    
    return _lists, _dicts

    